<a href="https://colab.research.google.com/github/mrafiwd/datasetNutrition/blob/main/Nutrition_KG_LLM_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knowledge Graph dan RAG untuk Data Nutrisi Makanan Indonesia
## Tujuan Proyek
Proyek ini bertujuan untuk membangun sebuah sistem tanya-jawab cerdas mengenai nutrisi makanan dengan alur kerja sebagai berikut:


1.   **Membangun Knowledge Graph** (KG): Mengubah data tabular (CSV) tentang nutrisi menjadi sebuah graf yang terstruktur di dalam database Neo4j menggunakan Large Language Model (LLM).
2.   **Implementasi RAG**: Mengembangkan sistem Retrieval-Augmented Generation (RAG) yang menggunakan KG tersebut sebagai basis pengetahuan utama.
3. **Hasil Akhir**: Menciptakan aplikasi interaktif yang mampu menjawab pertanyaan pengguna secara faktual berdasarkan data yang diambil langsung dari Knowledge Graph.






# **Notes**
Dataset yang digunakan dalam implementasi ini merupakan subset dari kumpulan data yang lebih besar. Kami mengurangi jumlah data yang diambil menjadi 200 row untuk menyesuaikan dengan batasan rate limit (15 permintaan per menit) yang berlaku pada API Google Gemini (via AI Studio), sehingga proses pembangunan knowledge graph dapat diselesaikan dalam waktu yang wajar untuk tujuan demonstrasi ini.

Berikut link dataset:
[https://www.kaggle.com/datasets/anasfikrihanif/indonesian-food-and-drink-nutrition-dataset](https://www.kaggle.com/datasets/anasfikrihanif/indonesian-food-and-drink-nutrition-dataset)

Jadi, dataset yang sudah disesuaikan dapat diakses pada:
[https://github.com/mrafiwd/datasetNutrition/blob/main/nutrition.csv](https://github.com/mrafiwd/datasetNutrition/blob/main/nutrition.csv)

In [ ]:
%pip uninstall google-generativeai google-ai-generativelanguage -y

Found existing installation: google-ai-generativelanguage 0.6.18
Uninstalling google-ai-generativelanguage-0.6.18:
  Successfully uninstalled google-ai-generativelanguage-0.6.18


In [ ]:
# 1. Instalasi Pustaka yang Diperlukan
# Perintah ini akan menginstal semua library yang dibutuhkan untuk proyek ini.
!pip install -U -q langchain langchain-google-genai langchain-experimental neo4j pandas requests tqdm langchain_neo4j

## Input your own credentials:
1. GOOGLE_API_KEY (use https://aistudio.google.com/apikey to generate gemini api key)
2. NEO4J_URI
3. NEO4J_USERNAME
4. NEO4J_PASSWORD

Untuk Kredensial Neo4j bisa didapatkan pada Connection Details dari Project Sandbox yang dibuat.

In [ ]:
# Sel 2: Impor dan Kredensial
import os
import getpass
import requests
import asyncio
import pandas as pd
from tqdm.auto import tqdm # Menggunakan tqdm.auto

# --- Impor ---
from langchain_core.documents import Document
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_neo4j import Neo4jGraph
from google.colab import userdata

# --- Impor tambahan untuk RAG ---
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Pengaturan Kredensial
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY') #using secrets

NEO4J_URI = "bolt://100.27.17.174:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "chip-prerequisite-sharpeners"

os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

# Inisialisasi Komponen Inti


1.   Menginisialisasi dua objek LLM terpisah: graph_creation_llm untuk pembuatan graf dan rag_llm untuk RAG, guna menghindari konflik konfigurasi.
2.   Menyiapkan LLMGraphTransformer yang bertugas mengubah teks menjadi struktur graf.
3. Menyiapkan objek Neo4jGraph sebagai konektor ke database.

In [ ]:
# Sel 3: Inisialisasi Komponen (Versi Definitif dengan LLM Terpisah)

# LLM #1: Khusus untuk membuat graf. Cukup pakai model yang cepat seperti flash.
graph_creation_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

llm_transformer = LLMGraphTransformer(
    llm=graph_creation_llm,
    allowed_nodes=["Food", "Nutrient", "Value", "Unit"],
    node_properties=["name", "calories", "protein", "carbohydrate", "fat"],
    allowed_relationships=["HAS_NUTRIENT", "HAS_VALUE", "HAS_UNIT"]
)

# Inisialisasi koneksi ke database Neo4j (tetap sama)
graph = Neo4jGraph()

print("✅ Komponen berhasil diinisialisasi dengan LLM terpisah.")

✅ Komponen berhasil diinisialisasi dengan LLM terpisah.


# Definisi Arsitektur dan Logika Inti


1.   **create_graph_if_needed()**: Berisi logika untuk menjalankan pipeline ETL (Extract, Transform, Load) dari file CSV ke database Neo4j untuk membangun Knowledge Graph.

In [ ]:
# Sel 4: Definisi Fungsi (Versi Baru)

# Import tqdm yang sesuai
from tqdm.auto import tqdm

# Definisi fungsi 'load_and_process_csv_from_url' tetap sama, tidak perlu diubah.
def load_and_process_csv_from_url(url, batch_size=5):
    try:
        df = pd.read_csv(url)
        print("Dataset CSV berhasil dimuat. Berikut 5 baris pertama:")
        print(df.head())
    except Exception as e:
        print(f"Error membaca file CSV dari URL: {e}")
        return []
    sentences = []
    for _, row in df.iterrows():
        row = row.fillna('')
        food_name = str(row.get('name', '')).strip()
        if food_name:
            calories = row.get('calories', 0)
            protein = row.get('proteins', 0)
            carbohydrate = row.get('carbohydrate', 0)
            fat = row.get('fat', 0)
            sentence = f"Makanan bernama {food_name} memiliki {calories} kalori, {protein} gram protein, {carbohydrate} gram karbohidrat, dan {fat} gram lemak."
            sentences.append(sentence)
    if not sentences:
        print("\n❌ Peringatan: Tidak ada kalimat yang berhasil dibuat. Periksa nama kolom di kode dan CSV.")
        return []
    batched_documents = []
    for i in range(0, len(sentences), batch_size):
        batch_content = "\n".join(sentences[i:i+batch_size])
        batched_documents.append(Document(page_content=batch_content))
    print(f"\nData berhasil diubah. Total kalimat: {len(sentences)}. Dibagi menjadi {len(batched_documents)} batch.")
    return batched_documents


async def process_documents_to_graph(documents):
    """
    Versi perbaikan: Menggunakan 'await' dengan benar di dalam perulangan for standar.
    """
    graph_documents = []
    # Gunakan perulangan for standar dengan progress bar dari tqdm
    for doc in tqdm(documents, desc="Mengubah dokumen menjadi graf..."):
        # 1. 'await' untuk mendapatkan hasil (sebuah list) dari coroutine
        graph_doc_result = await llm_transformer.aconvert_to_graph_documents([doc])

        # 2. Lakukan .extend() pada hasil yang sudah didapatkan
        graph_documents.extend(graph_doc_result)

        # 3. Tetap berikan jeda untuk rate limiting
        await asyncio.sleep(4)
    return graph_documents
# ==============================================================================


# Definisi fungsi 'main' juga tetap sama
async def create_graph_if_needed():
    github_csv_url = "https://raw.githubusercontent.com/mrafiwd/datasetNutrition/refs/heads/main/nutrition.csv"
    BATCH_SIZE = 3
    documents_to_process = load_and_process_csv_from_url(github_csv_url, batch_size=BATCH_SIZE)
    if documents_to_process:
        final_graph_documents = await process_documents_to_graph(documents_to_process)
        print("\n--- Hasil Ekstraksi Graf ---")
        if final_graph_documents:
            all_nodes = [node for doc in final_graph_documents for node in doc.nodes]
            all_relationships = [rel for doc in final_graph_documents for rel in doc.relationships]
            unique_nodes = list({node.id: node for node in all_nodes}.values())
            print(f"Total Node unik diekstrak: {len(unique_nodes)}")
            print(f"Total Relasi diekstrak: {len(all_relationships)}")

            print("\nMenyimpan graf ke database Neo4j...")
            # PERBAIKAN: Hapus argumen 'base_entity_label'
            graph.add_graph_documents(
                final_graph_documents,
                include_source=True
            )
            print("✅ Graf berhasil disimpan ke Neo4j.")
        else:
            print("❌ Tidak ada graf yang berhasil dibuat dari dokumen.")
    else:
        print("❌ Proses dihentikan karena tidak ada data yang bisa diproses menjadi kalimat.")

# Eksekusi
*   Sel di bawah memanggil **create_graph_if_needed** untuk mengisi database (apabila sudah membuat graf, tidak perlu dijalankan).
*   Sel berikutnya memulai sesi tanya-jawab interaktif dengan pengguna.

Menjalankan create_graph_if_needed untuk membuat knowledge graph

In [ ]:
# Anda bisa menjalankan pembuatan graf terlebih dahulu jika database kosong
await create_graph_if_needed() #Jika database sudah ada, line ini tidak perlu di-run

Dataset CSV berhasil dimuat. Berikut 5 baris pertama:
  id  calories  proteins   fat  carbohydrate                name  \
0  1     280.0       9.2  28.4           0.0                Abon   
1  2     513.0      23.7  37.0          21.3        Abon haruwan   
2  3       0.0       0.0   0.2           0.0           Agar-agar   
3  4      45.0       1.1   0.4          10.8  Akar tonjong segar   
4  5      37.0       4.4   0.5           3.8       Aletoge segar   

                                               image  
0  https://img-cdn.medkomtek.com/PbrY9X3ignQ8sVuj...  
1  https://img-global.cpcdn.com/recipes/cbf330fbd...  
2  https://res.cloudinary.com/dk0z4ums3/image/upl...  
3  https://images.tokopedia.net/img/cache/200-squ...  
4  https://nilaigizi.com/assets/images/produk/pro...  

Data berhasil diubah. Total kalimat: 191. Dibagi menjadi 64 batch.


Mengubah dokumen menjadi graf...:   0%|          | 0/64 [00:00<?, ?it/s]


--- Hasil Ekstraksi Graf ---
Total Node unik diekstrak: 210
Total Relasi diekstrak: 72

Menyimpan graf ke database Neo4j...
✅ Graf berhasil disimpan ke Neo4j.


# Definisi Arsitektur Graph Retriever dan RAG Chain
1.   **graph_retriever()**: Fungsi inti RAG. Fungsinya mengambil pertanyaan pengguna, mencari node :Food yang relevan di Neo4j menggunakan Full-Text Index, dan mengembalikan properti dari node tersebut sebagai konteks.
2. **rag_chain**: Merangkai retriever, prompt, dan rag_llm menjadi satu alur kerja tanya-jawab menggunakan LangChain Expression Language (LCEL).

In [ ]:
# Sel 5: Menyiapkan Retriever dari Knowledge Graph
# ==============================================================================
# Buat rantai (chain) kecil untuk mengekstrak kata kunci dari pertanyaan pengguna
keyword_extraction_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Your task is to extract the main food name or nutrient from the user's question. Only return the extracted keyword, nothing else.",
        ),
        ("human", "{question}"),
    ]
)
keyword_extractor_chain = keyword_extraction_prompt | graph_creation_llm | StrOutputParser()

def graph_retriever(question: str):
    """
    Versi Definitif: Menggunakan Full-Text Search Index Neo4j untuk pencarian
    yang akurat dan andal.
    """

    # Mengatur bahwa keyword yang digunakan adalah question langsung
    keyword = question
    # Query Cypher yang menggunakan Full-Text Index.
    # Ini jauh lebih kuat daripada CONTAINS atau STARTS WITH.
    cypher_query = """
    CALL db.index.fulltext.queryNodes("food_search_index", $keyword) YIELD node AS n, score
    WITH n, score ORDER BY score DESC LIMIT 1 // Ambil hasil teratas yang paling relevan
    MATCH (n)-[r]-(m)
    RETURN n, r, m
    """
    try:
        result = graph.query(cypher_query, params={"keyword": keyword})
        if not result:
             return "Konteks tidak ditemukan di Knowledge Graph untuk kata kunci ini."
        print(f"Konteks ditemukan untuk '{keyword}':\n{result}") # Tambahkan print untuk debugging
        return f"Konteks dari Knowledge Graph:\n{result}"
    except Exception as e:
        return f"Gagal mengambil data dari graf: {e}"

In [ ]:
# Sel 6: Membuat RAG Chain Lengkap
# ==============================================================================
# Template prompt untuk RAG
rag_prompt_template = """Anda adalah asisten AI yang ahli dalam data nutrisi makanan.
Jawab pertanyaan pengguna secara ringkas dan jelas HANYA berdasarkan konteks yang diberikan dari knowledge graph.
Jika konteks tidak menyediakan informasi yang cukup, katakan bahwa Anda tidak menemukan datanya.

Konteks dari Knowledge Graph:
{context}

Pertanyaan Pengguna:
{question}

Jawaban Anda:
"""

rag_prompt = ChatPromptTemplate.from_template(rag_prompt_template)

# Merangkai semua komponen menjadi satu RAG chain menggunakan LCEL
rag_chain = (
    {"context": graph_retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | graph_creation_llm
    | StrOutputParser()
)

# Contoh Pertanyaan yang Bisa Ditanyakan


1. "jelaskan tentang abon"
2. "bagaimana nutrisi untuk alpukat?"
3. "info gizi agar-agar"
4. "kalori pada apel"
5. "abon haruwan"

## **Notes**
Untuk makanan yang bisa ditanyakan terbatas hanya pada dataset yang terdapat di Github:
[https://github.com/mrafiwd/datasetNutrition/blob/main/nutrition.csv](https://github.com/mrafiwd/datasetNutrition/blob/main/nutrition.csv)

In [ ]:
# Sel 7: Eksekusi dan Pengujian
# ==============================================================================

# --- Sekarang Anda bisa mulai bertanya ---
print("\n--- Sistem Tanya-Jawab Nutrisi Berbasis Knowledge Graph ---")
print("Ketik 'exit' untuk keluar.")

while True:
    pertanyaan = input("\nSilakan bertanya tentang nutrisi makanan: ")
    if pertanyaan.lower() == 'exit':
        break
    if pertanyaan:
        # Memanggil RAG chain untuk mendapatkan jawaban
        jawaban = rag_chain.invoke(pertanyaan)
        print("\nJawaban:")
        print(jawaban)


--- Sistem Tanya-Jawab Nutrisi Berbasis Knowledge Graph ---
Ketik 'exit' untuk keluar.

Silakan bertanya tentang nutrisi makanan: lemak dan karbohidrat pada abon haruwan
Konteks ditemukan untuk 'lemak dan karbohidrat pada abon haruwan':
[{'n': {'fat': '37.0', 'id': 'Abon Haruwan', 'calories': '513.0', 'carbohydrate': '21.3', 'protein': '23.7'}, 'r': ({'id': '96518f7fac904b5772e8c7275d129c8e', 'text': 'Makanan bernama Abon memiliki 280.0 kalori, 9.2 gram protein, 0.0 gram karbohidrat, dan 28.4 gram lemak.\nMakanan bernama Abon haruwan memiliki 513.0 kalori, 23.7 gram protein, 21.3 gram karbohidrat, dan 37.0 gram lemak.\nMakanan bernama Agar-agar memiliki 0.0 kalori, 0.0 gram protein, 0.0 gram karbohidrat, dan 0.2 gram lemak.'}, 'MENTIONS', {'fat': '37.0', 'id': 'Abon Haruwan', 'calories': '513.0', 'carbohydrate': '21.3', 'protein': '23.7'}), 'm': {'id': '96518f7fac904b5772e8c7275d129c8e', 'text': 'Makanan bernama Abon memiliki 280.0 kalori, 9.2 gram protein, 0.0 gram karbohidrat, dan 2